# Importacoes

In [ ]:
import tensorflow as tf


In [ ]:
print(tf.__version__)


In [ ]:
import requests
import pandas as pd
import numpy as np


# Conectando com servidor

In [ ]:
diaInicio = '20230301'
diaFinal = '20230531'
idGrupo = '15'
idRegra = '139'
idTipo = 'null'
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreino?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(url)


In [ ]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)


In [ ]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')


In [ ]:
dfTratadoX = df.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape


In [ ]:
Y = np.stack(df['campoY'].values)
Y = Y.reshape(-1,1)
Y.shape


# Dados Validacao

In [ ]:
diaInicio = '20230531'
diaFinal = '20230531'
idGrupo = '15'
idRegra = '139'
idTipo = 'null'
urlValida = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoSaidas/listaParaTreino?diaNumInicio={}&diaNumFinal={}&idGrupoAcao={}&idRegraProjecao={}&idTipoExemplo={}'.format(diaInicio,diaFinal,idGrupo,idRegra,idTipo)


In [ ]:
print(urlValida)


In [ ]:
r = requests.get(urlValida)
j = r.json()
dfValida = pd.DataFrame.from_dict(j)


In [ ]:
dfValida.head()


In [ ]:
dfTratadoValidaX = dfValida.apply(trataLinhaX, axis='columns')
XValida = np.stack(dfTratadoValidaX.values)
XValida.shape


In [ ]:
YValida = np.stack(dfValida['campoY'].values)
YValida = YValida.reshape(-1,1)
YValida.shape


# Minha Rede

## Estrutra

In [ ]:
entrada_principal = tf.keras.layers.Input(name='entrada_principal',shape=(150,1))
lstm1_principal = tf.keras.layers.LSTM(name='lstm1_principal', units=64, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True) (entrada_principal)
flatten_principal = tf.keras.layers.Flatten(name='flatten_principal') (lstm1_principal)
dense_principal = tf.keras.layers.Dense(name='dense_principal', units=64, activation='relu')(flatten_principal)

entrada_secundaria = tf.keras.layers.Input(name='entrada_secundaria',shape=(600,1))
lstm1_secundaria = tf.keras.layers.LSTM(name='lstm1_secundaria', units=256, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True) (entrada_secundaria)
lstm2_secundaria = tf.keras.layers.LSTM(name='lstm2_secundaria', units=128, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True) (lstm1_secundaria)
lstm3_secundaria = tf.keras.layers.LSTM(name='lstm3_secundaria', units=64, activation='tanh', kernel_initializer=tf.keras.initializers.HeUniform(), return_sequences=True) (lstm2_secundaria)
flatten_secundaria = tf.keras.layers.Flatten(name='flatten_secundaria') (lstm3_secundaria)
dense_secundaria = tf.keras.layers.Dense(name='dense_secundaria', units=64, activation='relu')(flatten_secundaria)

concat = tf.keras.layers.Concatenate(name='concat')([dense_principal, dense_secundaria])
output = tf.keras.layers.Dense(name='output',units=1, activation='sigmoid')(concat)

model = tf.keras.Model(inputs=[entrada_principal, entrada_secundaria], outputs=[output])
model.summary()


## Compilacao

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy',  metrics=['accuracy'])


## Fit - Treino

In [ ]:

callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)
Ent1X2 = Entrada1.T
Ent1X2 = Ent1X2[np.newaxis,...]
Ent2X2 = Entrada2.T
Ent2X2 = Ent2X2[np.newaxis,...]

history = model.fit(x=[Ent1X2.T,Ent2X2.T],y=Y,epochs=80,verbose=2,callbacks=[callback])


## Salva Rede

In [ ]:
path='pesos/treino_rede1045.h5'
model.save(path)


# Enviando Rede

In [ ]:
!pip install paramiko


In [ ]:
import os
import paramiko


In [ ]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect("191.252.219.136", username="root", password="Digicom2004$")
sftp = ssh.open_sftp()
sftp.put("pesos/treino_rede1045.h5", "/root/executa-rede/redes/treino_rede1045.h5")
sftp.close()
ssh.close()


# Registra Treinamento

In [ ]:
#import requests
url = 'http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/executouTreinamento'
myobj = {'id': 1045}
x = requests.post(url, json = myobj)
print(x.text)
